In [20]:
import pandas as pd
#import torch
import numpy as np

from tqdm import tqdm

# Balance data for Task1

In [1]:
datapath = '../segments_final.xlsx'
df = pd.read_excel(datapath)

In [22]:
df['sum_req'] = df[df.columns[-9:]].sum(axis=1)
df['label'] = [1 if l > 0 else 0 for l in df['sum_req']]

In [23]:
df['label'].value_counts()

label
0    8478
1    2106
Name: count, dtype: int64

In [24]:
def get_stats(df, label_name = 'label'):    
    sum_per_project_req =df.groupby(['project',label_name])['id'].count().reset_index()
    sum_per_project = sum_per_project_req.groupby('project').sum().reset_index()
    stats_org_data = pd.merge(sum_per_project_req, sum_per_project, on = 'project')
    stats_org_data['perc'] = stats_org_data['id_x']/stats_org_data['id_y']
    return stats_org_data

In [25]:
stats_org_data = get_stats(df)

In [27]:
print(stats_org_data[stats_org_data['label_x'] ==0]['perc'].std(ddof=0), stats_org_data[stats_org_data['label_x'] ==1]['perc'].std(ddof=0))
print(stats_org_data[stats_org_data['label_x'] ==0]['perc'].mean(), stats_org_data[stats_org_data['label_x'] ==1]['perc'].mean())

0.15565022240481158 0.15565022240481158
0.7487478969475427 0.25125210305245715


In [28]:
label_name = 'label'

In [29]:
all_rows = []
for p in df['project'].unique():
    df_subset = df[df['project'] == p]
    values_counts = df_subset[label_name].value_counts()
    if max(values_counts) > round(min(values_counts) * 1.5):
        df_subset_major = df_subset[df_subset[label_name] == np.argmax(values_counts)]
        index_sample = np.random.choice(df_subset_major.index, round(min(values_counts) * 1.5))
        all_rows.extend(df_subset_major.loc[index_sample].to_dict('records'))
        all_rows.extend(df_subset[df_subset[label_name] == np.argmin(values_counts)].to_dict('records'))
    else:
        all_rows.extend(df_subset.to_dict('records'))
    

In [30]:
df_label_balanced =pd.DataFrame(all_rows)

In [31]:
df_label_balanced['label'].value_counts()

label
0    2990
1    2106
Name: count, dtype: int64

In [36]:
stats_balanced_data = get_stats(df_label_balanced)
stats_balanced_data[stats_balanced_data['label_x'] ==0]['perc'].mean()

0.5824210341600914

In [37]:
stats_balanced_data[stats_balanced_data['label_x'] ==0]['perc'].std(ddof=0)

0.04739905716363859

In [17]:
#df_label_balanced.to_excel('segments_balanced.xlsx', index=False)

# Balancing data for Task2

In [ ]:
datapath = '../segments_types_final.xlsx'

df = pd.read_excel(datapath)

In [5]:
df['label_type'].value_counts()

label_type
0    1375
1     352
2     295
Name: count, dtype: int64

In [6]:
avg_count_per_label = df.groupby(['project', 'label_type']).count().reset_index()[['project','label_type', 'id']].groupby('label_type').mean('id').to_dict()['id']

In [7]:
label_name = 'label_type'

In [3]:
all_rows = []
for p in df['project'].unique():
    df_subset = df[df['project'] == p]
    values_counts = df_subset[label_name].value_counts()
    for k, v in values_counts.items():
        if v > round(min(values_counts) * 2):
            new_value = round(max(avg_count_per_label[np.argmin(values_counts)], min(values_counts) * 2))
            df_subset_major = df_subset[df_subset[label_name] == k]
            index_sample = np.random.choice(df_subset_major.index, new_value)
            all_rows.extend(df_subset_major.loc[index_sample].to_dict('records'))
        else:
            all_rows.extend(df_subset[df_subset[label_name] == k].to_dict('records'))

In [13]:
avg_count_per_label = df.groupby(['project', 'label_type']).count().reset_index()[['project','label_type', 'id']].groupby('label_type').mean('id').reset_index()

In [15]:
stats_balanced_data = get_stats(pd.DataFrame(all_rows), 'label_type')
stats_org_data = get_stats(df, 'label_type')


In [16]:
print(stats_org_data[stats_org_data['label_type_x'] ==0]['perc'].std(ddof=0),
stats_org_data[stats_org_data['label_type_x'] ==1]['perc'].std(ddof=0),
stats_org_data[stats_org_data['label_type_x'] ==2]['perc'].std(ddof=0))

0.20442024265202238 0.2121353001970558 0.07190544714041365


In [17]:
print(stats_org_data[stats_org_data['label_type_x'] ==0]['perc'].mean(),
stats_org_data[stats_org_data['label_type_x'] ==1]['perc'].mean(),
stats_org_data[stats_org_data['label_type_x'] ==2]['perc'].mean())

0.6471024707705231 0.2064859351501615 0.1464115940793155


In [18]:
print(stats_balanced_data[stats_balanced_data['label_type_x'] ==0]['perc'].std(ddof=0),
stats_balanced_data[stats_balanced_data['label_type_x'] ==1]['perc'].std(ddof=0),
stats_balanced_data[stats_balanced_data['label_type_x'] ==2]['perc'].std(ddof=0))

0.09268008424814356 0.17541114956054796 0.12320727201275117


In [19]:
print(stats_balanced_data[stats_balanced_data['label_type_x'] ==0]['perc'].mean(),
stats_balanced_data[stats_balanced_data['label_type_x'] ==1]['perc'].mean(),
stats_balanced_data[stats_balanced_data['label_type_x'] ==2]['perc'].mean())

0.4205650962021405 0.2884209041316325 0.29101399966622704


In [11]:
pd.DataFrame(all_rows)['label_type'].value_counts()

label_type
0    457
1    308
2    291
Name: count, dtype: int64

In [14]:
#pd.DataFrame(all_rows).to_excel('segments_types_balanced.xlsx', index=False)